In [89]:
import streamlit as st
from langchain.memory import ChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory, 
                                                  ConversationSummaryMemory)
                                                  
from functions import *
import dotenv; dotenv.load_dotenv()
import os

In [20]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. If you do not then tell the human you do not know.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [32]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [33]:
conversation = ConversationChain(
            llm=llm,
            verbose=True,
        )

In [39]:
chain = (
    conversation
    | llm
    | StrOutputParser()
)

In [40]:
conversation.predict(input="Hello!")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello!
AI: Hello! How are you today?
Human: Hello!
AI: Hello again! Did you have a good day so far?
Human: Hello!
AI:

> Finished chain.


"Hello once more! Is there anything specific you'd like to talk about today?"

In [65]:
conversation.memory.buffer

"Human: Hello!\nAI: Hello! How are you today?\nHuman: Hello!\nAI: Hello again! Did you have a good day so far?\nHuman: Hello!\nAI: Hello once more! Is there anything specific you'd like to talk about today?\nHuman: Hello!\nAI: Hello again! It seems like you're saying hello a lot. Is there something on your mind that you'd like to discuss?"

In [66]:
pattern = r'(Human:.*?)(?=\nAI:|$)\n(AI:.*?)(?=\nHuman:|$)'

# Find all the pairs (Human: response, AI: response)
pairs = re.findall(pattern, conversation.memory.buffer)

# Output the result as a list of tuples
print(pairs)

[('Human: Hello!', 'AI: Hello! How are you today?'), ('Human: Hello!', 'AI: Hello again! Did you have a good day so far?'), ('Human: Hello!', "AI: Hello once more! Is there anything specific you'd like to talk about today?"), ('Human: Hello!', "AI: Hello again! It seems like you're saying hello a lot. Is there something on your mind that you'd like to discuss?")]


In [ ]:
def parse_memory(memory):
    pattern = r'(Human:.*?)(?=\nAI:|$)\n(AI:.*?)(?=\nHuman:|$)'
    return re.findall(pattern, memory)

In [71]:
message = ('Human: Hello!', 'AI: Hello! How are you today?')

In [95]:
history = parse_json('responses.json')
history

[('Hello!', 'Hello! How are you today?'),
 ('Hello!', 'Hello again! Did you have a good day so far?'),
 ('Hello!',
  "Hello once more! Is there anything specific you'd like to talk about today?"),
 ('Hello!',
  "Hello again! It seems like you're saying hello a lot. Is there something on your mind that you'd like to discuss?")]

In [96]:
chat_history = ChatMessageHistory()

for i in history:
    chat_history.add_user_message(i[0])
    chat_history.add_ai_message(i[1])

In [111]:
chat_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello again! Did you have a good day so far?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello once more! Is there anything specific you'd like to talk about today?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello again! It seems like you're saying hello a lot. Is there something on your mind that you'd like to discuss?", additional_kwargs={}, response_metadata={})])

In [80]:
# Specify the path where you want to save the JSON file
import json
file_path = 'responses.json'

# Write the list to the file as JSON
with open(file_path, 'w') as json_file:
    json.dump(pairs, json_file, indent=4)

print(f"List has been written to {file_path}")

List has been written to responses.json


In [55]:
conversation.memory.content

AttributeError: 'ConversationBufferMemory' object has no attribute 'content'

In [36]:
conversation.output_parser

TypeError: 'StrOutputParser' object is not callable

In [14]:
conversation.predict(input="Do you know what DNS is?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello!
AI: Hello! How are you today?
Human: Do you know what DNS is?
AI:

> Finished chain.


'Yes, DNS stands for Domain Name System. It is a system that translates domain names into IP addresses, allowing users to access websites using easy-to-remember domain names instead of complex numerical IP addresses. DNS servers are responsible for this translation process and help route internet traffic efficiently.'